In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options

browser = Browser('chrome')

# # Chrome options and preferences
# chrome_options = Options()
# chrome_options.add_argument("--blink-settings=imagesEnabled=false")

# # Chrome with options
# browser = Browser('chrome', options=chrome_options)


In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
cookie = browser.find_by_id('btn-cookie-allow').click()
# cookie.click()

In [4]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(1)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

0
1
2


In [5]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)
        

In [6]:
len(links)

167

In [7]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []


max_iteration2 = 20
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()
    
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        rating.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        rating.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        rating.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        rating.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        rating.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
#     try:
#         moretaste = soup.find('div', class_ = 'foodParings pip-info')
#         all_taste = moretaste.find_all('li')
#         taste = {}

#         for taste_record in all_taste:
#             taste_value1 = taste_record.find('div', class_ = 'value').text
#             taste_name = taste_record.find('div', class_ = 'label').text
#             taste_value = taste_value1.get('aria-label')
#             taste[taste_name] = taste_value
#         taste_list.append(taste)
#     except AttributeError:
#         taste_list.append('N/A')
    
    print(iteration2)
    iteration2 += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [11]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list
}

# # Add the details dictionary to the data dictionary
# data['Details'] = details_list
# # data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

# details_df = pd.json_normalize(df['Details'])
# # taste_df = pd.json_normalize(df['Taste'])

# df = pd.concat([df, details_df], axis = 1)
# # df2 = pd.concat([df1, taste_df], axis = 1)

# df.drop('Details', axis = 1, inplace = True)
# # df2.drop('Taste', axis = 1, inplace = True)

# # Display the dataframe Test
# df.head(20)

ValueError: All arrays must be of the same length